In [2]:
# !pip install minio
# !pip install boto3
# !pip install pyspark requests
# !pip install pyspark kafka-python


In [1]:
from pyspark.sql import SparkSession

minio_url = "http://minio:9000"
access_key = "minio"
secret_key = "SU2orange!"

spark = SparkSession.builder \
    .appName("MinioSpark") \
    .config("spark.hadoop.fs.s3a.endpoint", minio_url) \
    .config("spark.hadoop.fs.s3a.access.key", access_key)\
    .config("spark.hadoop.fs.s3a.secret.key", secret_key) \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.hadoop.fs.s3a.path.style.access", True)\
    .config("spark.hadoop.fs.s3a.fast.upload", True)\
    .config("spark.sql.streaming.kafka.useDeprecatedOffsetFetching", "true") \
    .config("spark.sql.streaming.checkpointLocation", "/tmp/spark/checkpoint")\
    .config("spark.sql.extensions", "org.apache.spark.sql.kafka010.KafkaSparkSessionExtension") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.2")\
    .getOrCreate()
sc = spark.sparkContext
sc.setLogLevel("ERROR")

:: loading settings :: url = jar:file:/usr/local/spark-3.1.2-bin-hadoop3.2/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-fd3e1c36-56fe-4d9b-afb1-f9a6a183d5f3;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.3.2 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.3.2 in central
	found org.apache.kafka#kafka-clients;2.8.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.32 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.2 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.3.2 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commons-pool2;2.11.1 in central
downloa

In [14]:
import boto3
from botocore.client import Config
from botocore.exceptions import ClientError
s3_client = boto3.client(
    's3',
    endpoint_url=minio_url,
    aws_access_key_id=access_key,
    aws_secret_access_key=secret_key,
    config=Config(signature_version='s3v4')
)
bucket1_name = "indexfund"
bucket2_name = "ibm"

In [15]:
def create_bucket(bucket_name):
    try:
        s3_client.head_bucket(Bucket=bucket_name)
        print(f"Bucket '{bucket_name}' already exists.")
    except ClientError as e:
        if e.response['Error']['Code'] == '404':
            try:
                s3_client.create_bucket(Bucket=bucket_name)
                print(f"Bucket '{bucket_name}' created successfully.")
            except Exception as e:
                print(f"Error creating bucket: {e}")
        else:
            print(f"Error checking bucket existence: {e}")


In [16]:
create_bucket(bucket1_name)
create_bucket(bucket2_name)

Bucket 'indexfund' already exists.
Bucket 'ibm' already exists.


In [17]:
from pyspark.sql.functions import col
import requests
import json

In [18]:
KAFKA_BROKER = 'localhost:9092'
KAFKA_TOPIC = 'IBM'
ALPHA_VANTAGE_API_KEY = '2NS9D8LV0S0DZVKB'
ALPHA_VANTAGE_URL = 'https://www.alphavantage.co/query'
ALPHA_VANTAGE_FUNCTION = 'TIME_SERIES_INTRADAY'
SYMBOL = 'IBM'

In [19]:
def fetch_data():
    params = {
        'function': ALPHA_VANTAGE_FUNCTION,
        'symbol': SYMBOL,
        'interval': '1min',
        'apikey': ALPHA_VANTAGE_API_KEY
    }
    response = requests.get(ALPHA_VANTAGE_URL, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"Error fetching data: {response.status_code}")


In [20]:
def transform_to_df(data):
    time_series = data.get('Time Series (1min)', {})
    rows = []
    for timestamp, values in time_series.items():
        row = {
            'timestamp': timestamp,
            'open': float(values.get('1. open', 0)),
            'high': float(values.get('2. high', 0)),
            'low': float(values.get('3. low', 0)),
            'close': float(values.get('4. close', 0)),
            'volume': int(values.get('5. volume', 0))
        }
        rows.append(row)
    return spark.createDataFrame(rows)

In [21]:
data = fetch_data()
data_frame = transform_to_df(data)

In [24]:
kafka_streaming_df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers",KAFKA_BROKER) \
    .option("subscribe", "test-topic") \
    .load()

AnalysisException: Failed to find data source: kafka. Please deploy the application as per the deployment section of "Structured Streaming + Kafka Integration Guide".

In [3]:
# minio_path = "s3a://sohamtestbucket/df.csv"

In [18]:
# df.coalesce(1).write.option("inferSchema",True).option("header",True).mode("overwrite").csv(minio_path)

In [21]:
# test = spark.read.option("inferSchema",True).option("header",True).csv(minio_path)

+----+---+
|Name|Age|
+----+---+
|John| 28|
|Jane| 35|
| Sam| 40|
+----+---+

